In [ ]:
!pip install keras_preprocessing

In [ ]:
import os
import urllib.request
import zipfile
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception

In [ ]:
# Unduh dan Ekstrak Dataset
data_url = 'https://drive.usercontent.google.com/download?id=1T2vteOAvBrj23MK3ry_ynLWli4_gNbqe&export=download&confirm=t'
urllib.request.urlretrieve(data_url, 'Datasets.zip')
with zipfile.ZipFile('Datasets.zip', 'r') as zip_ref:
    zip_ref.extractall('Datasets')

base_dir = 'Datasets'
train_path = os.path.join(base_dir, 'Dataset/train')
val_path = os.path.join(base_dir, 'Dataset/validation')
test_path = os.path.join(base_dir, 'Dataset/test')

In [ ]:
# ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.2,
    shear_range=0.3,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(299, 299),
    batch_size=64,
    class_mode='categorical'
)

val_datagen = ImageDataGenerator(rescale=1.0/255.0)
validation_generator = val_datagen.flow_from_directory(
    val_path,
    target_size=(299, 299),
    batch_size=64,
    class_mode='categorical'
)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(299, 299),
    batch_size=64,
    class_mode='categorical'
)


In [ ]:
# Xception Base Model
pre_trained_model = Xception(input_shape=(299, 299, 3),
                              include_top=False,
                              weights='imagenet')

# Bekukan semua layer kecuali 30 terakhir
for layer in pre_trained_model.layers[:-30]:
    layer.trainable = False
for layer in pre_trained_model.layers[-30:]:
    layer.trainable = True

last_layer_output = pre_trained_model.output
print(f'Output shape of the Xception base model: {last_layer_output.shape}')

In [ ]:
# Model Tambahan dan Training
x = layers.GlobalAveragePooling2D()(last_layer_output)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(3, activation='softmax')(x)

model = Model(pre_trained_model.input, x)

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['acc'])

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'Xception_model.h5',
    monitor='val_acc',
    save_best_only=True
)

history = model.fit(
    train_generator,
    epochs=120,
    validation_data=validation_generator,
    callbacks=[model_checkpoint]
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - acc: 0.5303 - loss: 0.9414

17/17 ━━━━━━━━━━━━━━━━━━━━ 134s 5s/step - acc: 0.5366 - loss: 0.9313 - val_acc: 0.7651 - val_loss: 0.7078
Epoch 2/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 67s 2s/step - acc: 0.7758 - loss: 0.5459 - val_acc: 0.6812 - val_loss: 1.6884
Epoch 3/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - acc: 0.8276 - loss: 0.4242 - val_acc: 0.7181 - val_loss: 1.3257
Epoch 4/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - acc: 0.8684 - loss: 0.3441

17/17 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - acc: 0.8686 - loss: 0.3441 - val_acc: 0.8557 - val_loss: 0.6203
Epoch 5/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - acc: 0.9192 - loss: 0.2600 - val_acc: 0.7953 - val_loss: 0.8034
Epoch 6/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - acc: 0.8954 - loss: 0.2891 - val_acc: 0.8423 - val_loss: 0.8745
Epoch 7/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.9114 - loss: 0.2537 - val_acc: 0.7215 - val_loss: 1.0568
Epoch 8/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - acc: 0.8920 - loss: 0.2671 - val_acc: 0.8121 - val_loss: 0.5208
Epoch 9/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - acc: 0.9392 - loss: 0.1704 - val_acc: 0.8188 - val_loss: 0.5659
Epoch 10/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - acc: 0.9279 - loss: 0.1684 - val_acc: 0.7685 - val_loss: 0.9334
Epoch 11/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - acc: 0.9494 - loss: 0.1846 - val_acc: 0.8188 - val_loss: 1.0008
Epoch 12/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - acc: 0.9568 - loss: 

17/17 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - acc: 0.9818 - loss: 0.0613 - val_acc: 0.8725 - val_loss: 0.7244
Epoch 20/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.9798 - loss: 0.0575 - val_acc: 0.8624 - val_loss: 0.9922
Epoch 21/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.9657 - loss: 0.0991 - val_acc: 0.7416 - val_loss: 1.7051
Epoch 22/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - acc: 0.9745 - loss: 0.0888 - val_acc: 0.7919 - val_loss: 1.0156
Epoch 23/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.9598 - loss: 0.1080 - val_acc: 0.8691 - val_loss: 0.5631
Epoch 24/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - acc: 0.9880 - loss: 0.0570 - val_acc: 0.8624 - val_loss: 0.5286
Epoch 25/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - acc: 0.9743 - loss: 0.0722 - val_acc: 0.8356 - val_loss: 0.7940
Epoch 26/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - acc: 0.9733 - loss: 0.0671 - val_acc: 0.8490 - val_loss: 0.6932
Epoch 27/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.9768 - l

17/17 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - acc: 0.9757 - loss: 0.0671 - val_acc: 0.8758 - val_loss: 0.5947
Epoch 29/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.9794 - loss: 0.0798 - val_acc: 0.8758 - val_loss: 0.5232
Epoch 30/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.9902 - loss: 0.0299 - val_acc: 0.8523 - val_loss: 1.0154
Epoch 31/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - acc: 0.9879 - loss: 0.0329 - val_acc: 0.8658 - val_loss: 0.9036
Epoch 32/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - acc: 0.9786 - loss: 0.0711 - val_acc: 0.8523 - val_loss: 0.8488
Epoch 33/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - acc: 0.9794 - loss: 0.0571 - val_acc: 0.8591 - val_loss: 0.7143
Epoch 34/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.9897 - loss: 0.0329 - val_acc: 0.8490 - val_loss: 1.0357
Epoch 35/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - acc: 0.9863 - loss: 0.0442

17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.9863 - loss: 0.0440 - val_acc: 0.8792 - val_loss: 0.9349
Epoch 36/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - acc: 0.9886 - loss: 0.0292 - val_acc: 0.8691 - val_loss: 0.8369
Epoch 37/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.9850 - loss: 0.0458 - val_acc: 0.8054 - val_loss: 1.0359
Epoch 38/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.9874 - loss: 0.0481 - val_acc: 0.8591 - val_loss: 0.6675
Epoch 39/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - acc: 0.9947 - loss: 0.0250 - val_acc: 0.8188 - val_loss: 0.8763
Epoch 40/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - acc: 0.9851 - loss: 0.0509 - val_acc: 0.8389 - val_loss: 0.9486
Epoch 41/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.9851 - loss: 0.0378 - val_acc: 0.8289 - val_loss: 1.0489
Epoch 42/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.9905 - loss: 0.0223 - val_acc: 0.8356 - val_loss: 1.1847
Epoch 43/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - acc: 0.9843 - l

17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.9929 - loss: 0.0360 - val_acc: 0.8926 - val_loss: 0.5322
Epoch 55/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.9872 - loss: 0.0379 - val_acc: 0.8121 - val_loss: 1.3834
Epoch 56/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.9909 - loss: 0.0299 - val_acc: 0.8557 - val_loss: 0.9187
Epoch 57/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - acc: 0.9904 - loss: 0.0221 - val_acc: 0.8456 - val_loss: 0.8543
Epoch 58/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.9881 - loss: 0.0311 - val_acc: 0.8658 - val_loss: 1.0624
Epoch 59/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.9912 - loss: 0.0294 - val_acc: 0.8356 - val_loss: 1.0626
Epoch 60/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.9888 - loss: 0.0359 - val_acc: 0.8826 - val_loss: 0.9106
Epoch 61/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.9938 - loss: 0.0218 - val_acc: 0.8691 - val_loss: 0.8369
Epoch 62/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.9866 - l

17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.9928 - loss: 0.0147 - val_acc: 0.8960 - val_loss: 0.6857
Epoch 65/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.9936 - loss: 0.0141 - val_acc: 0.8725 - val_loss: 0.8314
Epoch 66/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - acc: 0.9919 - loss: 0.0239 - val_acc: 0.8591 - val_loss: 1.2944
Epoch 67/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.9898 - loss: 0.0271 - val_acc: 0.8725 - val_loss: 0.9756
Epoch 68/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.9848 - loss: 0.0399 - val_acc: 0.8423 - val_loss: 0.8599
Epoch 69/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.9807 - loss: 0.0594 - val_acc: 0.8523 - val_loss: 0.5833
Epoch 70/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - acc: 0.9921 - loss: 0.0328 - val_acc: 0.8624 - val_loss: 0.5923
Epoch 71/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.9940 - loss: 0.0150 - val_acc: 0.8725 - val_loss: 0.6406
Epoch 72/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - acc: 0.9839 - l

17/17 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - acc: 0.9961 - loss: 0.0142 - val_acc: 0.9027 - val_loss: 0.7815
Epoch 109/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.9937 - loss: 0.0159 - val_acc: 0.8423 - val_loss: 1.2115
Epoch 110/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.9888 - loss: 0.0297 - val_acc: 0.8389 - val_loss: 0.7910
Epoch 111/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.9907 - loss: 0.0196 - val_acc: 0.8658 - val_loss: 0.8452
Epoch 112/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.9954 - loss: 0.0123 - val_acc: 0.8557 - val_loss: 0.6404
Epoch 113/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.9985 - loss: 0.0082 - val_acc: 0.8826 - val_loss: 0.6698
Epoch 114/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.9965 - loss: 0.0080 - val_acc: 0.8624 - val_loss: 0.6682
Epoch 115/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - acc: 0.9956 - loss: 0.0118 - val_acc: 0.8557 - val_loss: 0.8288
Epoch 116/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.

In [ ]:
train_path = os.path.join(base_dir, "Dataset/train")
val_path = os.path.join(base_dir, "Dataset/validation")
test_path = os.path.join(base_dir, "Dataset/test")

train_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(299, 299),
    batch_size=128,
    class_mode='categorical'
)

validation_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
validation_generator = validation_datagen.flow_from_directory(
    val_path,
    target_size=(299, 299),
    batch_size=128,
    class_mode='categorical'
)

test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(299, 299),
    batch_size=128,
    class_mode='categorical'
)

model = tf.keras.models.load_model('Xception_model.h5')

train_loss, train_acc = model.evaluate(train_generator, verbose=1)
val_loss, val_acc = model.evaluate(validation_generator, verbose=1)
test_loss, test_acc = model.evaluate(test_generator, verbose=1)

print(f"Train Accuracy: {train_acc:.4f}, Loss: {train_loss:.4f}")
print(f"Validation Accuracy: {val_acc:.4f}, Loss: {val_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}, Loss: {test_loss:.4f}")

In [ ]:
import matplotlib.pyplot as plt

# Ambil riwayat dari training
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

# Plot Akurasi
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs, acc, 'r', label='Training Accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation Accuracy')
plt.title('Accuracy - Xception Model')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Loss - Xception Model')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow import keras
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical

In [ ]:
def load_and_process_image(image_path, target_size=(299, 299)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img) / 255.0
    return img_array

def load_test_data(test_dir, class_names):
    data = []
    labels = []
    class_mapping = {name: idx for idx, name in enumerate(class_names)}
    for class_name in class_names:
        class_path = os.path.join(test_dir, class_name)
        if not os.path.isdir(class_path):
            print(f"Warning: Folder {class_path} not found.")
            continue
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            try:
                data.append(load_and_process_image(image_path))
                labels.append(class_mapping[class_name])
            except Exception as e:
                print(f"Error loading image {image_path}: {e}")
    return np.array(data), np.array(labels)

In [ ]:
test_data_dir = val_path
class_names = sorted(os.listdir(test_data_dir))
X_test, y_test = load_test_data(test_data_dir, class_names)

model = keras.models.load_model('Xception_model.h5')

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(to_categorical(y_test, num_classes=len(class_names)), axis=1)

conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
class_report = classification_report(y_true_classes, y_pred_classes, target_names=class_names)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix - Xception Model (Validation)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.tight_layout()
plt.show()

print("\nClassification Report (Validation Data):")
print(class_report)

In [ ]:
test_data_dir = train_path
class_names = sorted(os.listdir(test_data_dir))
X_test, y_test = load_test_data(test_data_dir, class_names)

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(to_categorical(y_test, num_classes=len(class_names)), axis=1)

conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
class_report = classification_report(y_true_classes, y_pred_classes, target_names=class_names)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix - Xception Model (Training)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.tight_layout()
plt.show()

print("\nClassification Report (Training Data):")
print(class_report)

In [ ]:
test_data_dir = test_path
class_names = sorted(os.listdir(test_data_dir))
X_test, y_test = load_test_data(test_data_dir, class_names)

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(to_categorical(y_test, num_classes=len(class_names)), axis=1)

conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
class_report = classification_report(y_true_classes, y_pred_classes, target_names=class_names)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix - Xception Model (Test)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.tight_layout()
plt.show()

print("\nClassification Report (Test Data):")
print(class_report)